In [2]:
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.datasets import reuters

import numpy as np
import plotly.graph_objects as go

In [3]:
num_words = 10000
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words = num_words)

2113536/2110848 [==============================] - 0s 0us/step


In [16]:
np.array(train_data[0])

array([   1,    2,    2,    8,   43,   10,  447,    5,   25,  207,  270,
          5, 3095,  111,   16,  369,  186,   90,   67,    7,   89,    5,
         19,  102,    6,   19,  124,   15,   90,   67,   84,   22,  482,
         26,    7,   48,    4,   49,    8,  864,   39,  209,  154,    6,
        151,    6,   83,   11,   15,   22,  155,   11,   15,    7,   48,
          9, 4579, 1005,  504,    6,  258,    6,  272,   11,   15,   22,
        134,   44,   11,   15,   16,    8,  197, 1245,   90,   67,   52,
         29,  209,   30,   32,  132,    6,  109,   15,   17,   12])

In [4]:
# split the data to train and validation datasets
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(train_data, train_labels, test_size = .25)

In [5]:
# helper to decode the sequence
def decoder(seq):
  word_index = reuters.get_word_index()
  r_word_index = dict([(val,key) for (key,val) in word_index.items()])
  decoded_seq = " ".join([r_word_index.get(i - 3, "?") for i in seq])
  return decoded_seq
# helper to vectorize the input data
def vectorize(seqs, ndim):
  encoded_seqs = np.zeros((len(seqs), ndim))
  for i, seq in enumerate(seqs):
    encoded_seqs[i,seq] = 1.
  return encoded_seqs

In [6]:
# vectorize the input data
x_val = vectorize(x_val,num_words)
x_train = vectorize(x_train,num_words)
x_test = vectorize(test_data,num_words)
y_test = test_labels

In [7]:
# vectorize the targets
K = len(set(y_train)) # number of classes

In [8]:
batch_size = 128
epochs = 20
lr = 0.001
units = [64, 128]
activations = ['relu', 'relu']
t_fc_layers = 2


model = models.Sequential()
model.add(layers.Input((num_words,)))
for l in range(t_fc_layers):
  model.add(layers.Dense(units[l], activation = activations[l]))
model.add(layers.Dense(K, activation = 'softmax'))

# compile the model with RMSprop optimizer.
model.compile(optimizer = optimizers.RMSprop(lr=lr),
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

# train the model
r = model.fit(x_train, y_train, validation_data = (x_val, y_val), batch_size = batch_size, epochs = epochs)
print("Average train loss:",np.mean(r.history['loss']))
print("Average test loss:",np.mean(r.history['val_loss']))
print('Average train accuracy:',np.mean(r.history['accuracy']))
print('Average test accuracy:',np.mean(r.history['val_accuracy']))
# evaluate the test data
print('Evaluation score')
model.evaluate(x_test, y_test)

Epoch 1/20
53/53 [==============================] - 1s 21ms/step - loss: 1.7933 - accuracy: 0.6265 - val_loss: 1.2815 - val_accuracy: 0.7026
Epoch 2/20
53/53 [==============================] - 1s 18ms/step - loss: 0.9410 - accuracy: 0.7910 - val_loss: 1.0532 - val_accuracy: 0.7694
Epoch 3/20
53/53 [==============================] - 1s 18ms/step - loss: 0.6126 - accuracy: 0.8688 - val_loss: 0.9671 - val_accuracy: 0.7916
Epoch 4/20
53/53 [==============================] - 1s 19ms/step - loss: 0.4029 - accuracy: 0.9129 - val_loss: 0.9290 - val_accuracy: 0.8032
Epoch 5/20
53/53 [==============================] - 1s 19ms/step - loss: 0.2805 - accuracy: 0.9406 - val_loss: 0.9779 - val_accuracy: 0.8037
Epoch 6/20
53/53 [==============================] - 1s 19ms/step - loss: 0.2115 - accuracy: 0.9501 - val_loss: 1.0300 - val_accuracy: 0.7930
Epoch 7/20
53/53 [==============================] - 1s 19ms/step - loss: 0.1755 - accuracy: 0.9611 - val_loss: 1.0215 - val_accuracy: 0.8037
Epoch 8/20
53

[1.5757631063461304, 0.7809438705444336]

In [9]:
history_dict = r.history

In [10]:
# Plotly kullanarak antrenman esnasında hesaplanan lossu ve accuracyi grafiğe dökelim
fig = go.Figure()
fig.update_layout(title='Losses',
                  xaxis_title='Epochs',
                  yaxis_title = 'Loss')
fig.add_trace(go.Scatter(x=list(range(epochs)), y = history_dict['loss'], mode = 'lines+markers', name="training loss"))
fig.add_trace(go.Scatter(x=list(range(epochs)), y = history_dict['val_loss'], mode = 'lines+markers', name = 'validation loss'))
fig.show()

fig.data = [] # fig objesinde bulunan yukarıdaki traceler data attribute unda tutulur. 
              # Bunu sıfırlayalım ki aynı objeyi tekrar kullanarak accuracyi grafiğe dökebilelim
fig.update_layout(title='Accuracies',
                  yaxis_title = 'Accuracy')
fig.add_trace(go.Scatter(x=list(range(epochs)), y = history_dict['accuracy'], mode = 'lines+markers', name="training acc"))
fig.add_trace(go.Scatter(x=list(range(epochs)), y = history_dict['val_accuracy'], mode = 'lines+markers', name="validation acc"))
fig.show()

In [11]:
preds = model.predict(x_test) # modeli test verisi üzerinde kullanıp tahminleri alalım
preds = np.argmax(preds,1) # bir sample için elde edilen olasılık dağılımının maksimumunun bulunduğu indexleri alalım

In [12]:
# Kategorileri aşağıdaki github issue sundan aldım
# src: https://github.com/keras-team/keras/issues/12072
labels = ['cocoa','grain','veg-oil','earn','acq','wheat','copper','housing','money-supply',
   'coffee','sugar','trade','reserves','ship','cotton','carcass','crude','nat-gas',
   'cpi','money-fx','interest','gnp','meal-feed','alum','oilseed','gold','tin',
   'strategic-metal','livestock','retail','ipi','iron-steel','rubber','heat','jobs',
   'lei','bop','zinc','orange','pet-chem','dlr','gas','silver','wpi','hog','lead']

In [20]:
# yazının sağa doğru taşmaması için pretty print ile bir genişlik belirleyelim
import pprint
pp = pprint.PrettyPrinter(width = 80, compact = True)

# artık boyutları da yapıları da aynı olduğundan doğru olan/olmayan sınıflandırmaları ayıklayabiliriz
misclassified_idx = np.where(preds != y_test)[0]
correct_idx = np.where(preds == y_test)[0]

# yanlış sınıflandırılanlardan rastgele bir örnek
r = np.random.choice(misclassified_idx)
review = test_data[r]
review = decoder(review)
print("Yanlış sınıflandırma örneği\n\n")
print(f"Tahmin: {labels[preds[r]]}, Gerçek: {labels[test_labels[r]]}")
pp.pprint(review)
# doğru sınıflandırmalardan rastgele bir örnek
r = np.random.choice(correct_idx)
review = test_data[r]
review = decoder(review)
print("\n\nDoğru sınıflandırma örneği\n\n")
print(f"Sınıf: {labels[preds[r]]}")
pp.pprint(review)

Yanlış sınıflandırma örneği


Tahmin: acq, Gerçek: earn
('? lt hoechst celanese corp recently formed from the merger of the celanese '
 'corp with american hoechst corp a unit of lt hoechst ag said juergen ? '
 "former member of ? ag's management board has been named chairman and chief "
 'executive officer of hoechst celanese the chemical company also said it '
 'named dieter ? ? formerly president of american hoechst vice chairman while '
 'ernest drew former group vice president of celanese has been named president '
 'and chief operating officer hoechst celanese said richard ? also formerly a '
 'group vice president of celanese has been appointed senior executive vice '
 'president of the new company and harry ? former executive vice president and '
 'chief financial officer of american hoechst has been named executive vice '
 'president and chief financial officer the company also said the former '
 "celanese corp's fiber operations will operate separately as celanese ? inc a "
 